In [181]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
from itertools import chain
from collections import Counter
import gensim
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.preprocessing import label_binarize
from porter import PorterStemmer

In [5]:
cleaned = pd.read_csv("../data/cleaned.csv")
images = pd.read_csv("../data/images.csv")

In [202]:
cleaned.head(5)

,rating,cleaned,bigram,tweet,created_at,replies_count,retweets_count,likes_count,img_url
0,9,new view love art presentation perspective lig...,newview viewlove loveart artpresentation prese...,New view. Love the art. Presentation. Perspect...,2021-03-17 16:24:35 EDT,3,2,62,https://pbs.twimg.com/media/EwtT4E7UUAAgt5w.jpg
1,9,st paddy’s day edition great wall color maps a...,stpaddy paddy’ ’s sday dayedition editiongreat...,Room Rater St Paddy’s Day Edition. Great wall ...,2021-03-17 15:25:51 EDT,2,2,57,https://pbs.twimg.com/media/EwtGbzTUYAAxDiu.jpg
2,10,doors dublin dingus-the stuff dreams made nice...,doorsdublin dublindingus-the dingus-thestuff s...,Doors of Dublin. The Dingus-the stuff that dre...,2021-03-17 15:19:53 EDT,7,4,123,https://pbs.twimg.com/media/EwtFESHVEAEYlzv.jpg
3,9,good tight set up love blue art flag widen sho...,goodtight tightset setup uplove loveblue bluea...,Good tight set up. Love the blue. Art. Flag. W...,2021-03-17 13:46:47 EDT,1,1,60,https://pbs.twimg.com/media/EwsvwlOU8AQ5lOY.jpg
4,0,dark never escape stank wrong about everything...,darknever neverescape escapestank stankwrong w...,Dark. Will never escape the stank. Wrong about...,2021-03-17 13:11:17 EDT,35,38,846,https://pbs.twimg.com/media/Ewsnoq5U8AMxxAg.jpg


In [7]:
cleaned.iloc[0]["cleaned"]

'new view love art presentation perspective light/lighting widen shot slightly @davidjollyfl'

In [8]:
cleaned.iloc[7]["cleaned"]

'book club love art lighting flowers arkansas traveler @iamsophianelson'

In [97]:
# lowest rating
low_rating = cleaned[cleaned['rating'] == 0]
low_rating.head(3) # yields row 4, 20, 28

# semi low rating
semi_low_rating = cleaned[cleaned['rating'] == 3]
semi_low_rating.head(3) # yields row 75, 135, 153

# middle rating
middle_rating = cleaned[cleaned['rating'] == 5]
middle_rating.head(3) # yields rows 169, 240, 245

#semi middle rating
semi_middle_rating = cleaned[cleaned['rating'] == 7]
semi_middle_rating.head(3) # yields rows 37, 45, 69

# semi high rating
semi_high_rating = cleaned[cleaned['rating'] == 8]
semi_high_rating.head(3) # yields rows 8, 9, 17

# highest rating
highest_rating = cleaned[cleaned['rating'] == 10]
highest_rating.head(3) # yields rows 2, 5, 7


,rating,cleaned,bigram,tweet,created_at,replies_count,retweets_count,likes_count,img_url
2,10,doors dublin dingus-the stuff dreams made nice...,doorsdublin dublindingus-the dingus-thestuff s...,Doors of Dublin. The Dingus-the stuff that dre...,2021-03-17 15:19:53 EDT,7,4,123,https://pbs.twimg.com/media/EwtFESHVEAEYlzv.jpg
5,10,st paddy’s day update @philiprucker green tie,stpaddy paddy’ ’s sday dayupdate update@ @phil...,Room Rater St Paddy’s Day Update. @PhilipRucke...,2021-03-17 12:34:31 EDT,10,12,620,https://pbs.twimg.com/media/EwsfN-yVoAAcVk8.jpg
7,10,book club love art lighting flowers arkansas t...,bookclub clublove loveart artlighting lighting...,Room Rater Book Club. Love the art. Lighting. ...,2021-03-17 11:49:52 EDT,1,4,87,https://pbs.twimg.com/media/EwsVABRVgAEJGvE.jpg


In [8]:
len(cleaned)

12619

## **ADDITIONAL PREPROCESSING:**
    * Removing user tags (starting with @)
    * Stemming each cleaned word in tweet

In [83]:
preprocessed = cleaned[['rating','cleaned']]

In [84]:
preprocessed.head(10)

,rating,cleaned
0,9,new view love art presentation perspective lig...
1,9,st paddy’s day edition great wall color maps a...
2,10,doors dublin dingus-the stuff dreams made nice...
3,9,good tight set up love blue art flag widen sho...
4,0,dark never escape stank wrong about everything...
5,10,st paddy’s day update @philiprucker green tie
6,6,sun set ups tough raise camera late christmas ...
7,10,book club love art lighting flowers arkansas t...
8,8,ducks packers big plant well composed sports s...
9,8,well composed set up good spacing lower camera...


In [85]:
def remove_tags(tweet):
    if pd.isnull(tweet):
        return []
    #print(tweet)
    tokens = tweet.split(' ')
    tokens = list(filter(lambda x: '@' not in x, tokens))
    return tokens

def stemWords(tokens):
    result = []
    if tokens is None:
        return tokens
    for token in tokens:
        if token.isalpha():
            stem = PorterStemmer()
            stemmed_word = stem.stem(token, 0, len(token) - 1)
            result.append(stemmed_word)
        else: 
            result.append(token)

    return result

def split(tweet):
    if pd.isnull(tweet):
        return []
    tokens = tweet.split(' ')
    return tokens


In [86]:
preprocessed['tags_removed'] = preprocessed['cleaned'].apply(lambda x : remove_tags(x))
preprocessed['stemmed'] = preprocessed['tags_removed'].apply(lambda x : stemWords(x))
preprocessed['cleaned'] = preprocessed['cleaned'].apply(lambda x : split(x))

/usr/local/anaconda3/envs/nvfel/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/anaconda3/envs/nvfel/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/anaconda3/envs/nvfel/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [87]:
preprocessed

,rating,cleaned,tags_removed,stemmed
0,9,"[new, view, love, art, presentation, perspecti...","[new, view, love, art, presentation, perspecti...","[new, view, love, art, present, perspect, ligh..."
1,9,"[st, paddy’s, day, edition, great, wall, color...","[st, paddy’s, day, edition, great, wall, color...","[st, paddy’s, dai, edit, great, wall, color, m..."
2,10,"[doors, dublin, dingus-the, stuff, dreams, mad...","[doors, dublin, dingus-the, stuff, dreams, mad...","[door, dublin, dingus-the, stuff, dream, made,..."
3,9,"[good, tight, set, up, love, blue, art, flag, ...","[good, tight, set, up, love, blue, art, flag, ...","[good, tight, set, up, love, blue, art, flag, ..."
4,0,"[dark, never, escape, stank, wrong, about, eve...","[dark, never, escape, stank, wrong, about, eve...","[dark, never, escap, stank, wrong, about, ever..."
...,...,...,...,...
12614,8,"[blue, works, not, just, books, plus, good, us...","[blue, works, not, just, books, plus, good, us...","[blue, work, not, just, book, plu, good, us, s..."
12615,2,"[“i’ll, just, put, sweatpants”, skype, rooms]","[“i’ll, just, put, sweatpants”, skype, rooms]","[“i’ll, just, put, sweatpants”, skype, room]"
12616,3,"[books, too, dark, little, way, personal, style]","[books, too, dark, little, way, personal, style]","[book, too, dark, littl, wai, person, style]"
12617,4,"[books, always, must, little, too, obvious, su...","[books, always, must, little, too, obvious, su...","[book, alwai, must, littl, too, obviou, such, ..."


In [15]:
all_words = list(chain(*preprocessed['stopwords']))

In [16]:
len(all_words)

140437

# **Utilizing LDA Topic Modeling**

**Only LDA Topic Model:** 
   1. Fits data into LDA to get topic models, topic model is trained
   2. Topic model is applied back to training reviews + each review gets topic
      distribution/probabilities
   3. Topic probabilities are used as features for rating prediction + fitted
      into linear regression model


## Fitting Only LDA

In [88]:
from gensim.models.ldamodel import LdaModel
from gensim.corpora import Dictionary
# preprocessed has columns: cleaned, tags_removed, stemmed

### Creating a Bag of Words for Cleaned + Stemmed

In [122]:
# Creating bag of words

# cleaned column
token_dict_cleaned = Dictionary(preprocessed.cleaned)
bag_of_words_cleaned = [token_dict_cleaned.doc2bow(tweet) for tweet in preprocessed['cleaned']]

# stemmed
token_dict_stemmed = Dictionary(preprocessed.stemmed)
bag_of_words_stemmed = [token_dict_stemmed.doc2bow(tweet) for tweet in preprocessed['stemmed']]

### Fitting Cleaned + Stemmed data into LDA model with 5 topics

In [125]:
# Fitting data into LDA (5 TOPICS)

# cleaned
lda_5_cleaned = LdaModel(bag_of_words_cleaned, 
                    num_topics = 5,
                    id2word = token_dict_cleaned,
                    random_state = 1, 
                    passes = 10)

#print(lda_5_cleaned.show_topics())

# stemmed
lda_5_stemmed = LdaModel(bag_of_words_stemmed, 
                    num_topics = 5,
                    id2word = token_dict_stemmed,
                    random_state = 1, 
                    passes = 10)

lda_5_stemmed.show_topics()

[(0,
  '0.016*"it’s" + 0.013*"not" + 0.012*"better" + 0.010*"like" + 0.010*"go" + 0.009*"get" + 0.009*"make" + 0.008*"don’t" + 0.008*"do" + 0.008*"covid"'),
 (1,
  '0.018*"skype" + 0.016*"room" + 0.012*"updat" + 0.010*"dai" + 0.010*"histor" + 0.009*"new" + 0.009*"pineappl" + 0.008*"flower" + 0.008*"happi" + 0.007*"on"'),
 (2,
  '0.021*"hostag" + 0.019*"video" + 0.012*"point" + 0.012*"it’s" + 0.010*"not" + 0.010*"on" + 0.009*"like" + 0.009*"book" + 0.008*"updat" + 0.008*"thing"'),
 (3,
  '0.048*"art" + 0.035*"love" + 0.027*"light" + 0.023*"plant" + 0.023*"add" + 0.023*"good" + 0.022*"book" + 0.020*"nice" + 0.020*"well" + 0.018*"up"'),
 (4,
  '0.036*"love" + 0.021*"art" + 0.018*"light" + 0.015*"plant" + 0.012*"colour" + 0.010*"depth" + 0.010*"game" + 0.010*"great" + 0.010*"lamp" + 0.009*"pillow"')]

### Fitting Cleaned + Stemmed data into LDA model with 10 topics

In [126]:
# Fitting data into LDA (10 TOPICS)

# cleaned
lda_10_cleaned = LdaModel(bag_of_words_cleaned, 
                    num_topics = 10,
                    id2word = token_dict_cleaned,
                    random_state = 1, 
                    passes = 10)

#print(lda_5_cleaned.show_topics())

# stemmed
lda_10_stemmed = LdaModel(bag_of_words_stemmed, 
                    num_topics = 10,
                    id2word = token_dict_stemmed,
                    random_state = 1, 
                    passes = 10)

lda_10_stemmed.show_topics()

[(0,
  '0.022*"dai" + 0.022*"covid" + 0.018*"cabin" + 0.016*"pleas" + 0.016*"happi" + 0.016*"you’re" + 0.014*"canada" + 0.012*"can" + 0.011*"birthdai" + 0.010*"3"'),
 (1,
  '0.026*"updat" + 0.019*"histor" + 0.019*"than" + 0.018*"pineappl" + 0.015*"flower" + 0.012*"get" + 0.012*"new" + 0.012*"thank" + 0.011*"live" + 0.011*"beam"'),
 (2,
  '0.031*"too" + 0.024*"not" + 0.020*"like" + 0.020*"it’s" + 0.016*"help" + 0.012*"hostag" + 0.011*"dark" + 0.011*"if" + 0.010*"go" + 0.009*"ok"'),
 (3,
  '0.055*"up" + 0.054*"book" + 0.047*"well" + 0.035*"nice" + 0.034*"good" + 0.034*"set" + 0.023*"lit" + 0.016*"point" + 0.016*"art" + 0.013*"shelf"'),
 (4,
  '0.026*"violat" + 0.024*"cord" + 0.016*"solid" + 0.015*"expect" + 0.013*"vibe" + 0.013*"oh" + 0.013*"music" + 0.010*"not" + 0.010*"minor" + 0.009*"fan"'),
 (5,
  '0.039*"hostag" + 0.036*"video" + 0.019*"deduct" + 0.018*"done" + 0.016*"best" + 0.014*"not" + 0.014*"try" + 0.014*"much" + 0.012*"big" + 0.011*"nail"'),
 (6,
  '0.071*"skype" + 0.064*"room

### Fitting Cleaned + Stemmed data into LDA model with 20 topics

In [93]:
# Fitting data into LDA (20 TOPICS)
lda_20 = LdaModel(bag_of_words_cleaned, 
                    num_topics = 20,
                    id2word = token_dict,
                    random_state = 1, 
                    passes = 10)

lda_20.show_topics()

[(10,
  '0.101*"love" + 0.094*"art" + 0.076*"great" + 0.036*"depth" + 0.034*"lighting" + 0.021*"plant" + 0.021*"flowers" + 0.020*"lamp" + 0.017*"chair" + 0.015*"blue"'),
 (14,
  '0.054*"see" + 0.031*"so" + 0.029*"can" + 0.028*"space" + 0.026*"like" + 0.023*"more" + 0.018*"make" + 0.018*"would" + 0.017*"art" + 0.017*"fix"'),
 (2,
  '0.090*"it’s" + 0.038*"always" + 0.035*"bad" + 0.032*"succulent" + 0.026*"way" + 0.024*"still" + 0.021*"top" + 0.020*"being" + 0.016*"dog" + 0.015*"position"'),
 (16,
  '0.080*"out" + 0.043*"not" + 0.029*"ok" + 0.029*"crop" + 0.026*"he’s" + 0.023*"that’s" + 0.022*"ceiling" + 0.018*"doesn’t" + 0.017*"music" + 0.017*"sure"'),
 (7,
  '0.049*"lovely" + 0.030*"than" + 0.023*"elegant" + 0.021*"cabinet" + 0.019*"open" + 0.019*"collection" + 0.019*"yellow" + 0.019*"nailed" + 0.018*"start" + 0.017*"update"'),
 (3,
  '0.025*"there’s" + 0.024*"1" + 0.021*"we’ll" + 0.020*"we’ve" + 0.017*"@jheil" + 0.016*"seen" + 0.015*"yet" + 0.015*"someone" + 0.014*"help" + 0.014*"best"

### For each cleaned + stemmed data, apply both 5 and 10 topic models to each tweet, output 5 topic and 10 topic probabilities for each tweet

In [201]:
def extract_topic_probabilities_cleaned(tokens, num_topics):
    bow = token_dict_cleaned.doc2bow(tokens)
    if num_topics == 10: 
        topic_probs = lda_10_cleaned.get_document_topics(bow)
    if num_topics == 5:
        topic_probs = lda_5_cleaned.get_document_topics(bow)
        
    probs = []
        
    probs_dict = dict(topic_probs)
        
    for i in range (0, num_topics):
        if i not in probs_dict: 
            probs.append(0)
        else: 
            probs.append(probs_dict[i])
    
    return probs

def extract_topic_probabilities_stemmed(tokens, num_topics):
    bow = token_dict_stemmed.doc2bow(tokens)
    if num_topics == 10: 
        topic_probs = lda_10_stemmed.get_document_topics(bow)
    if num_topics == 5:
        topic_probs = lda_5_stemmed.get_document_topics(bow)
        
    probs = []
        
    probs_dict = dict(topic_probs)
        
    for i in range (0, num_topics):
        if i not in probs_dict: 
            probs.append(0)
        else: 
            probs.append(probs_dict[i])
    
    return probs
   
extract_topic_probabilities_cleaned(processed, 10)

[0.010069694,
 0.66924024,
 0.010069694,
 0.25019646,
 0.010071237,
 0.010070478,
 0.010069693,
 0.010069954,
 0.010070965,
 0.010071561]

### Adding topic probabilities as columns to data

In [134]:
preprocessed['lda_10_probs_cleaned'] = preprocessed['cleaned'].apply(lambda x : extract_topic_probabilities_cleaned(x, 10))
preprocessed['lda_5_probs_cleaned'] = preprocessed['cleaned'].apply(lambda x : extract_topic_probabilities_cleaned(x, 5))

/usr/local/anaconda3/envs/nvfel/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/anaconda3/envs/nvfel/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [135]:
preprocessed['lda_10_probs_stemmed'] = preprocessed['stemmed'].apply(lambda x : extract_topic_probabilities_stemmed(x, 10))
preprocessed['lda_5_probs_stemmed'] = preprocessed['stemmed'].apply(lambda x : extract_topic_probabilities_stemmed(x, 5))

In [205]:
preprocessed.head(5)

,rating,cleaned,tags_removed,stemmed,lda_10_probs,lda_5_probs,lda_10_probs_cleaned,lda_5_probs_cleaned,lda_10_probs_stemmed,lda_5_probs_stemmed
0,9,"[new, view, love, art, presentation, perspecti...","[new, view, love, art, presentation, perspecti...","[new, view, love, art, present, perspect, ligh...","[0, 0.6853188, 0, 0.24769051, 0, 0, 0, 0, 0, 0]","[0.11110465, 0.8379315, 0.016782343, 0.0173785...","[0, 0.68527937, 0, 0.24772996, 0, 0, 0, 0, 0, 0]","[0.11110278, 0.8379478, 0.016782334, 0.0173642...","[0, 0, 0, 0.14467387, 0, 0, 0, 0, 0.7753759, 0]","[0.018320857, 0.018461967, 0.018315086, 0.9264..."
1,9,"[st, paddy’s, day, edition, great, wall, color...","[st, paddy’s, day, edition, great, wall, color...","[st, paddy’s, dai, edit, great, wall, color, m...","[0, 0.11529674, 0, 0.57690215, 0, 0, 0, 0.2611...","[0.013348684, 0.31573728, 0.0128906, 0.5755992...","[0, 0.11548184, 0, 0.5767406, 0, 0, 0, 0.26113...","[0.013347856, 0.31579176, 0.012890519, 0.57554...","[0, 0.31566647, 0, 0, 0, 0, 0, 0, 0.6309494, 0]","[0.013394717, 0.3650373, 0.013496481, 0.594382..."
2,10,"[doors, dublin, dingus-the, stuff, dreams, mad...","[doors, dublin, dingus-the, stuff, dreams, mad...","[door, dublin, dingus-the, stuff, dream, made,...","[0.19799763, 0.2285615, 0, 0, 0, 0, 0.12228879...","[0.7600076, 0.1963269, 0.014439243, 0.01468350...","[0.19799131, 0.22857866, 0, 0, 0, 0, 0.1222571...","[0.75999737, 0.19633839, 0.014439249, 0.014682...","[0, 0.057735033, 0, 0.54084694, 0, 0.1300493, ...","[0.10219681, 0.19941911, 0.014534805, 0.584886..."
3,9,"[good, tight, set, up, love, blue, art, flag, ...","[good, tight, set, up, love, blue, art, flag, ...","[good, tight, set, up, love, blue, art, flag, ...","[0, 0, 0, 0.60546404, 0.33275887, 0, 0, 0, 0, 0]","[0.015479421, 0.01595111, 0.015447192, 0.93746...","[0, 0, 0, 0.6054711, 0.3327518, 0, 0, 0, 0, 0]","[0.015479418, 0.015946027, 0.015447189, 0.9374...","[0, 0, 0, 0.4312952, 0, 0, 0, 0, 0.50202626, 0]","[0.016847817, 0.01667914, 0.016725928, 0.93293..."
4,0,"[dark, never, escape, stank, wrong, about, eve...","[dark, never, escape, stank, wrong, about, eve...","[dark, never, escap, stank, wrong, about, ever...","[0, 0, 0, 0, 0, 0, 0.4000559, 0.38573247, 0.15...","[0.018459236, 0.018510584, 0.77921337, 0.01831...","[0, 0, 0, 0, 0, 0, 0.39992207, 0.38575116, 0.1...","[0.018459152, 0.018509388, 0.7792248, 0.018318...","[0.01018724, 0.010193234, 0.2553968, 0.3359198...","[0.38353115, 0.5552803, 0.020458056, 0.0202665..."


### Creating dataframe for each number of topics and preprocessing pair, each dataframe has rating (y) and columns for each topic probability

In [206]:
# 10 TOPICS: CLEANED
lda_features_df_10_cleaned = preprocessed[['rating', 'lda_10_probs_cleaned']]
lda_features_df_10_cleaned[['0','1', '2', '3', '4', '5', '6', '7', '8', '9']] = pd.DataFrame(lda_features_df_10_cleaned.lda_10_probs_cleaned.tolist(), index=lda_features_df_10_cleaned.index)
lda_features_df_10_cleaned = lda_features_df_10_cleaned[['rating', '0','1', '2', '3', '4', '5', '6', '7', '8', '9']]
lda_features_df_10_cleaned

# 10 TOPICS: STEMMED
lda_features_df_10_stemmed = preprocessed[['rating', 'lda_10_probs_stemmed']]
lda_features_df_10_stemmed[['0','1', '2', '3', '4', '5', '6', '7', '8', '9']] = pd.DataFrame(lda_features_df_10_stemmed.lda_10_probs_stemmed.tolist(), index=lda_features_df_10_stemmed.index)
lda_features_df_10_stemmed = lda_features_df_10_stemmed[['rating', '0','1', '2', '3', '4', '5', '6', '7', '8', '9']]
lda_features_df_10_stemmed

# 5 TOPICS: CLEANED
lda_features_df_5_cleaned = preprocessed[['rating', 'lda_5_probs_cleaned']]
lda_features_df_5_cleaned[['0','1', '2', '3', '4']] = pd.DataFrame(lda_features_df_5_cleaned.lda_5_probs_cleaned.tolist(), index=lda_features_df_5_cleaned.index)
lda_features_df_5_cleaned = lda_features_df_5_cleaned[['rating', '0','1', '2', '3', '4']]
lda_features_df_5_cleaned

# 5 TOPICS: STEMMED
lda_features_df_5_stemmed = preprocessed[['rating', 'lda_5_probs_stemmed']]
lda_features_df_5_stemmed[['0','1', '2', '3', '4']] = pd.DataFrame(lda_features_df_5_stemmed.lda_5_probs_stemmed.tolist(), index=lda_features_df_5_stemmed.index)
lda_features_df_5_stemmed = lda_features_df_5_stemmed[['rating', '0','1', '2', '3', '4']]
lda_features_df_5_stemmed.head(5)

/usr/local/anaconda3/envs/nvfel/lib/python3.7/site-packages/pandas/core/frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,rating,0,1,2,3,4
0,9,0.018321,0.018462,0.018315,0.926455,0.018447
1,9,0.013395,0.365037,0.013496,0.594382,0.013690
2,10,0.102197,0.199419,0.014535,0.584886,0.098963
3,9,0.016848,0.016679,0.016726,0.932938,0.016809
4,0,0.383531,0.555280,0.020458,0.020267,0.020464


In [207]:
lda_features_df_5 = preprocessed[['rating', 'lda_5_probs']]
lda_features_df_5[['0','1', '2', '3', '4']] = pd.DataFrame(lda_features_df_5.lda_5_probs.tolist(), index=lda_features_df_5.index)
lda_features_df_5 = lda_features_df_5[['rating', '0','1', '2', '3', '4']]
lda_features_df_5.head(5)

/usr/local/anaconda3/envs/nvfel/lib/python3.7/site-packages/pandas/core/frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,rating,0,1,2,3,4
0,9,0.111105,0.837932,0.016782,0.017379,0.016803
1,9,0.013349,0.315737,0.012891,0.575599,0.082424
2,10,0.760008,0.196327,0.014439,0.014684,0.014543
3,9,0.015479,0.015951,0.015447,0.937465,0.015657
4,0,0.018459,0.018511,0.779213,0.018319,0.165498


### Fitting Logistic Regression Model on above dataframes, features (X) = topic probabilities, output (y) = rating

In [216]:
# (10 TOPICS)

# CLEANED

train_size = int(0.1 * len(lda_features_df_10_cleaned))
train = lda_features_df_10_cleaned[:train_size]
test = lda_features_df_10_cleaned[train_size:]

X_all = lda_features_df_10_cleaned[['0','1', '2', '3', '4', '5', '6', '7', '8', '9']]
y_all = lda_features_df_10_cleaned['rating']

X_train = train[['0','1', '2', '3', '4', '5', '6', '7', '8', '9']]
y_train = train['rating']

X_test = test[['0','1', '2', '3', '4', '5', '6', '7', '8', '9']]
y_test = test['rating']

labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

logm_10_cleaned = LogisticRegression()
logm_10_cleaned.fit(X_train, y_train)

print("CLEANED SCORES")

print("10 cleaned training acc: ", logm_10_cleaned.score(X_train, y_train))
print("10 cleaned testing acc: ", logm_10_cleaned.score(X_test, y_test))

y_pred = logm_10_cleaned.predict(X_test)
print("10 cleaned testing F1 score: ", f1_score(y_test, y_pred, average = 'macro'))

y_test_binarized = label_binarize(list(y_test), classes = labels)
y_preds_binarized = label_binarize(list(y_pred), classes = labels)
print("10 cleaned testing roc score: ", roc_auc_score(y_test_binarized, y_preds_binarized, average = 'macro', multi_class = 'ovo'))

# STEMMED

train_size = int(0.1 * len(lda_features_df_10_stemmed))
train = lda_features_df_10_stemmed[:train_size]
test = lda_features_df_10_stemmed[train_size:]

X_all = lda_features_df_10_stemmed[['0','1', '2', '3', '4', '5', '6', '7', '8', '9']]
y_all = lda_features_df_10_stemmed['rating']

X_train = train[['0','1', '2', '3', '4', '5', '6', '7', '8', '9']]
y_train = train['rating']

X_test = test[['0','1', '2', '3', '4', '5', '6', '7', '8', '9']]
y_test = test['rating']

logm_10_stemmed = LogisticRegression()
logm_10_stemmed.fit(X_train, y_train)

print("STEMMED SCORES")

print("10 stemmed training acc: ", logm_10_stemmed.score(X_train, y_train))
print("10 stemmed testing acc: ", logm_10_stemmed.score(X_test, y_test))

y_pred = logm_10_stemmed.predict(X_test)
print("10 stemmed testing F1 score: ", f1_score(y_test, y_pred, average = 'macro'))

y_test_binarized = label_binarize(list(y_test), classes = labels)
y_preds_binarized = label_binarize(list(y_pred), classes = labels)
print("10 stemmed testing roc score: ", roc_auc_score(y_test_binarized, y_preds_binarized, average = 'macro', multi_class = 'ovo'))



CLEANED SCORES
10 cleaned training acc:  0.44329896907216493
10 cleaned testing acc:  0.2841169219933087
10 cleaned testing F1 score:  0.029497076962185495
10 cleaned testing roc score:  0.5055781987019304
STEMMED SCORES
10 stemmed training acc:  0.44885011895321175
10 stemmed testing acc:  0.29961260785349536
10 stemmed testing F1 score:  0.03895414416761144
10 stemmed testing roc score:  0.5124778559464753


In [217]:
# fitting logistic regression model on training data (5 TOPICS)

# CLEANED

train_size = int(0.1 * len(lda_features_df_5_cleaned))
train = lda_features_df_5_cleaned[:train_size]
test = lda_features_df_5_cleaned[train_size:]

X_all = lda_features_df_5_cleaned[['0','1', '2', '3', '4']]
y_all = lda_features_df_5_cleaned['rating']

X_train = train[['0','1', '2', '3', '4']]
y_train = train['rating']

X_test = test[['0','1', '2', '3', '4']]
y_test = test['rating']

logm_5_cleaned = LogisticRegression()
logm_5_cleaned.fit(X_train, y_train)

print("CLEANED SCORES")

print("5 cleaned training score: ", logm_5_cleaned.score(X_train, y_train))
print("5 cleaned testing score: ", logm_5_cleaned.score(X_test, y_test))

y_pred = logm_5_cleaned.predict(X_test)
print("5 cleaned testing F1 score: ", f1_score(y_test, y_pred, average = 'macro'))

y_test_binarized = label_binarize(list(y_test), classes = labels)
y_preds_binarized = label_binarize(list(y_pred), classes = labels)
print("5 cleaned testing roc score: ", roc_auc_score(y_test_binarized, y_preds_binarized, average = 'macro', multi_class = 'ovo'))

# STEMMED

train_size = int(0.1 * len(lda_features_df_5_stemmed))
train = lda_features_df_5_stemmed[:train_size]
test = lda_features_df_5_stemmed[train_size:]

X_all = lda_features_df_5_stemmed[['0','1', '2', '3', '4']]
y_all = lda_features_df_5_stemmed['rating']

X_train = train[['0','1', '2', '3', '4']]
y_train = train['rating']

X_test = test[['0','1', '2', '3', '4']]
y_test = test['rating']

logm_5_stemmed = LogisticRegression()
logm_5_stemmed.fit(X_train, y_train)

print("STEMMED SCORES")

print("5 stemmed training score: ", logm_5_stemmed.score(X_train, y_train))
print("5 stemmed testing score: ", logm_5_stemmed.score(X_test, y_test))

y_pred = logm_5_stemmed.predict(X_test)
print("5 stemmed testing F1 score: ", f1_score(y_test, y_pred, average = 'macro'))

y_test_binarized = label_binarize(list(y_test), classes = labels)
y_preds_binarized = label_binarize(list(y_pred), classes = labels)
print("5 stemmed testing roc score: ", roc_auc_score(y_test_binarized, y_preds_binarized, average = 'macro', multi_class = 'ovo'))

CLEANED SCORES
5 cleaned training score:  0.4274385408406027
5 cleaned testing score:  0.2900158478605388
5 cleaned testing F1 score:  0.030602484476379446
5 cleaned testing roc score:  0.5072406203665009
STEMMED SCORES
5 stemmed training score:  0.43536875495638383
5 stemmed testing score:  0.30198978693431944
5 stemmed testing F1 score:  0.033609750498231734
5 stemmed testing roc score:  0.5108904819148777
